# A Custom Fully Convolutional Networks For Time Series Classification

### Importing Three Different Time Series Datasets by using sktime's `load_UCR_UEA_dataset` 
#### About the datasets:
* First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.
* Second dataset is called **ItalyPowerDemand**. It has 2 classes, It is of type Sensor, and it has one dimension.
* Third dataset is called **BME**. It has 3 classes, It is of type Simulated, and it has one dimension.

In [ ]:
# ------ Importing necessary packages!
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# load_UCR_UEA_dataset
from sktime.datasets import load_UCR_UEA_dataset

In [2]:
# ------ Importing Time Series Datasets!
# 2 Classes, 1D, Image
X_train_1, y_train_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="train", return_type="numpy2D")
X_test_1, y_test_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="test", return_type="numpy2D")

# 2 Classes, 1D, Sensor 
X_train_2, y_train_2 = load_UCR_UEA_dataset(name= "ItalyPowerDemand", split="train", return_type="numpy2D")
X_test_2, y_test_2 = load_UCR_UEA_dataset(name="ItalyPowerDemand", split="test", return_type="numpy2D")

# 3Classes, 1D, Simulated
X_train_3, y_train_3 = load_UCR_UEA_dataset(name = "BME", split="train", return_type="numpy2D")
X_test_3, y_test_3 = load_UCR_UEA_dataset(name = "BME", split="test", return_type="numpy2D")

In [3]:
print("Dataset 1:", X_train_1.shape, X_test_1.shape)
print("Dataset 2:", X_train_2.shape, X_test_2.shape)
print("Dataset 3:", X_train_3.shape, X_test_3.shape)

Dataset 1: (600, 80) (276, 80)
Dataset 2: (67, 24) (1029, 24)
Dataset 3: (30, 128) (150, 128)


We have different datasets, which means we must have a function that can take in different datasets with different length.  

The plain baseline is a **Fully Convolutional Networks** which is build by stacking three convolution blocks with the filter sizes {128,256, 128} in each block.  
* Unlike the MCNN and MC-CNN, We exclude any pooling operation.  
* The basic block is a convolutional layer followed by a batch normalization layer and a ReLU activation layer.  
* The convolution operation is fulfilled by three 1-D kernels with the sizes {8, 5, 3} without striding.
* Batch normalization is applied to speed up the convergence speed and help improve generalization. 
* After the convolution blocks, the features are fed into a global average pooling layer instead of a fully connected layer, which largely reduces the number of weights. The final label is produced by a softmax layer


The FCN Model is then trained with **Adam** with learning rate 0.001, ρ = 0.9, β1 = 0.9, β2 = 0.999 and e = 1e − 8!  

The loss function for the model is **categorical cross entropy**! With **accuracy** as metrics!

In [19]:
# We are having different dataset with different types and sizes!
def create_fcn_model(input_shape, num_classes):
    fcn_model = tf.keras.Sequential([
        keras.layers.Conv1D(filters=128, kernel_size=8, input_shape=input_shape),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv1D(filters=256, kernel_size=5),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv1D(filters=128, kernel_size=3),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    fcn_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.001, epsilon=1e-8),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return fcn_model

Defining **callbacks** parameters for fine tuning!  
Defining **label encoder** for converting data to numerical ones.

In [18]:
le = LabelEncoder()
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]

Creating training sets by using **train_test_split()** to split the data into random train and test subsets!  
* test_size = 0.2! 
    * 20% of the data will be used for the valiation/test set and the remaining 80% will be used for the training set!
* random_state=42!
    * Ensuring that the splits that generates are reproducible!

In [17]:
# --------- Creating and training FCN on the FIRST dataset ----------- 
X_train_FCN_1, X_val_FCN_1, y_train_FCN_1, y_val_FCN_1 = train_test_split(X_train_1, y_train_1, test_size=0.2, random_state=42)
# --------- Creating and training FCN on the SECOND dataset ----------- 
X_train_FCN_2, X_val_FCN_2, y_train_FCN_2, y_val_FCN_2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)
# ------------- Creating and training FCN on the THIRD dataset ------------
X_train_FCN_3, X_val_FCN_3, y_train_FCN_3, y_val_FCN_3 = train_test_split(X_train_3, y_train_3, test_size=0.2, random_state=42)

##### Preparing the training sets for the models or training!

In [29]:
# --------- First Training set, TYPE=Image ------------
X_train_FCN_1, X_val_FCN_1, y_train_FCN_1, y_val_FCN_1 = train_test_split(X_train_1, y_train_1, test_size=0.2, random_state=42)
# One-hot encode the labels
y_train_FCN_1 = tf.keras.utils.to_categorical(y_train_FCN_1, num_classes=np.unique(y_train_FCN_1).shape[0])
y_val_FCN_1 = tf.keras.utils.to_categorical(y_val_FCN_1, num_classes=np.unique(y_train_FCN_1).shape[0])
# Reshaping the dataset (batch_size, steps)
X_train_FCN_1 = X_train_FCN_1.reshape(X_train_FCN_1.shape[0], X_train_FCN_1.shape[1], -1)
X_val_FCN_1 = X_val_FCN_1.reshape(X_val_FCN_1.shape[0], X_val_FCN_1.shape[1], -1)

# --------- Second Training set, TYPE=Sensor ----------- 
X_train_FCN_2, X_val_FCN_2, y_train_FCN_2, y_val_FCN_2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)
# Converting to int.
y_train_FCN_2 = le.fit_transform(y_train_FCN_2) #Converting to int
y_val_FCN_2 = le.transform(y_val_FCN_2) #Converting to int
# One-hot encode the labels
y_train_FCN_2 = tf.keras.utils.to_categorical(y_train_FCN_2, num_classes=int(np.max(y_train_FCN_2) + 1))
y_val_FCN_2 = tf.keras.utils.to_categorical(y_val_FCN_2, num_classes= int(np.max(y_train_FCN_2) + 1))
# Reshaping the dataset (batch_size, steps)
X_train_FCN_2 = X_train_FCN_2.reshape(X_train_FCN_2.shape[0], X_train_FCN_2.shape[1], -1)
X_val_FCN_2 = X_val_FCN_2.reshape(X_val_FCN_2.shape[0], X_train_FCN_2.shape[1], -1)

# ------------- Third Training set, TYPE=Simulated ------------- #
X_train_FCN_3, X_val_FCN_3, y_train_FCN_3, y_val_FCN_3 = train_test_split(X_train_3, y_train_3, test_size=0.2, random_state=42)
# Converting to int.
y_train_FCN_3 = le.fit_transform(y_train_FCN_3) #Converting to int
y_val_FCN_3 = le.transform(y_val_FCN_3) #Converting to int
# Subtract 1 from the labels to make them start from 0
y_train_FCN_3 = y_train_FCN_3 - 1
y_val_FCN_3 = y_val_FCN_3 - 1
# One-hot encode the labels
y_train_FCN_3 = tf.keras.utils.to_categorical(y_train_FCN_3, num_classes=int(np.max(y_train_FCN_3) + 1))
y_val_FCN_3 = tf.keras.utils.to_categorical(y_val_FCN_3, num_classes=int(np.max(y_train_FCN_3) + 1))
# Reshaping the dataset (batch_size, steps)
X_train_FCN_3 = X_train_FCN_3.reshape(X_train_FCN_3.shape[0], -1)
X_val_FCN_3 = X_val_FCN_3.reshape(X_val_FCN_3.shape[0], -1)

##### Slicing and shuffling the training sets!

In [30]:
# ------------ First Training Set! ---------------
# Convert the numpy arrays to a tf.data.Dataset
train_dataset_FCN_1 = tf.data.Dataset.from_tensor_slices((X_train_FCN_1, y_train_FCN_1))
val_dataset_FCN_1 = tf.data.Dataset.from_tensor_slices((X_val_FCN_1, y_val_FCN_1))
# Shuffle, batch, and prefetch the dataset
train_dataset_FCN_1 = train_dataset_FCN_1.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_FCN_1 = val_dataset_FCN_1.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# ------------ Second Training Set! ---------------
# Convert the numpy arrays to a tf.data.Dataset
train_dataset_FCN_2 = tf.data.Dataset.from_tensor_slices((X_train_FCN_2, y_train_FCN_2))
val_dataset_FCN_2 = tf.data.Dataset.from_tensor_slices((X_val_FCN_2, y_val_FCN_2))
# Shuffle, batch, and prefetch the dataset
train_dataset_FCN_2 = train_dataset_FCN_2.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_FCN_2 = val_dataset_FCN_2.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# ------------ Third Training Set! ---------------
# Convert the numpy arrays to a tf.data.Dataset
train_dataset_FCN_3 = tf.data.Dataset.from_tensor_slices((X_train_FCN_3, y_train_FCN_3))
val_dataset_FCN_3 = tf.data.Dataset.from_tensor_slices((X_val_FCN_3, y_val_FCN_3))
# Shuffle, batch, and prefetch the dataset
train_dataset_FCN_3 = train_dataset_FCN_3.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_FCN_3 = val_dataset_FCN_3.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

#### Creating and Fitting the custom MLP model for the first dataset!
* Training the model on the first dataset!

In [ ]:
# Create the model
model_FCN_1 = create_fcn_model(X_train_FCN_1.shape[1:], np.unique(y_train_FCN_1).shape[0])
# Train the model
model_FCN_1.fit(train_dataset_FCN_1, epochs=10, validation_data=val_dataset_FCN_1, callbacks=callbacks)

##### Testing the model on unseen data!
* Pre-Processing The Test Data
* Evaluating the Custom FCN model on the **first** dataset!
    * First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.

In [14]:
# Prepare the test data
X_test_FCN_11 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], -1)
y_test_FCN_11 = tf.keras.utils.to_categorical(y_test_1, num_classes=np.unique(y_train_FCN_1).shape[0])

# Convert the numpy arrays to a tf.data.Dataset
test_dataset_FCN_1 = tf.data.Dataset.from_tensor_slices((X_test_FCN_11, y_test_FCN_11))
# Batch and prefetch the dataset
test_dataset_FCN_1 = test_dataset_FCN_1.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# Evaluate the model on the test data
test_loss, test_accuracy = model_FCN_1.evaluate(test_dataset_FCN_1)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

5/5 [==============================] - 1s 35ms/step - loss: 0.6876 - accuracy: 0.5833
Test loss: 0.6875571608543396, Test accuracy: 0.5833333134651184


#### Creating and Fitting the custom FCN model for the second dataset!
* Training the model on the second dataset!

In [33]:
# Creating the model
model_FCN_2 = create_fcn_model(X_train_FCN_2.shape[1:], np.unique(y_train_FCN_2).shape[0])
# Train the model WITHOUT callbacks
model_FCN_2.fit(train_dataset_FCN_2, epochs=10, validation_data=val_dataset_FCN_2)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6413 - accuracy: 0.5849 - val_loss: 0.7163 - val_accuracy: 0.2857
Epoch 2/10
1/1 [==============================] - 0s 55ms/step - loss: 0.6408 - accuracy: 0.5849 - val_loss: 0.7128 - val_accuracy: 0.2857
Epoch 3/10
1/1 [==============================] - 0s 46ms/step - loss: 0.6402 - accuracy: 0.5849 - val_loss: 0.7091 - val_accuracy: 0.2857
Epoch 4/10
1/1 [==============================] - 0s 69ms/step - loss: 0.6396 - accuracy: 0.5849 - val_loss: 0.7055 - val_accuracy: 0.2857
Epoch 5/10
1/1 [==============================] - 0s 54ms/step - loss: 0.6390 - accuracy: 0.5849 - val_loss: 0.7021 - val_accuracy: 0.2857
Epoch 6/10
1/1 [==============================] - 0s 53ms/step - loss: 0.6384 - accuracy: 0.5849 - val_loss: 0.6986 - val_accuracy: 0.2857
Epoch 7/10
1/1 [==============================] - 0s 54ms/step - loss: 0.6379 - accuracy: 0.5849 - val_loss: 0.6955 - val_accuracy: 0.4286
Epoch 8/10
1/1 [=============